In [ ]:
using Pkg
Pkg.activate("../../environments/refined-delaunay-for-flow-problems/");

In [ ]:
using LinearAlgebra
using Random
using SparseArrays
using Statistics

using AbstractPlotting
using CairoMakie
using Distances
using JuMP
using LightGraphs
using Parameters
using SCIP
using Triangle

In [ ]:
# create instance data
Random.seed!(0);

const N = 7
const WIDTH = 500
const HEIGHT = 300

x = 0.95 * WIDTH * rand(N)
y = 0.95 * HEIGHT * rand(N)
points = [x y];

In [ ]:
function make_scene(width=WIDTH, height=HEIGHT)
    return Scene(resolution=(width, height), show_axis=false, scale_plot=false)
end

In [ ]:
function draw_points!(points; markersize=4, color=:black)
    scatter!(points[:, 1], points[:, 2], markersize=markersize, color=color)
end

In [ ]:
make_scene()
draw_points!(points)

In [ ]:
struct Triangulation
    points::Matrix{Float64}   # n x 2
    edges::Matrix{Int64}      # m x 2
    triangles::Matrix{Int64}  # t x 3
    keep_edges::Vector{Int64} # k
end

Triangulation(p, e, t) = Triangulation(p, e, t, [])

In [ ]:
function unique_edges(triangles)
    set = Set()
    for t in 1:size(triangles, 1)
        triangle = triangles[t, :]
        push!(set, min(triangle[[1, 2]], triangle[[2, 1]]))
        push!(set, min(triangle[[2, 3]], triangle[[3, 2]]))
        push!(set, min(triangle[[1, 3]], triangle[[3, 1]]))
    end
    return hcat(sort(collect(set))...)'
end

In [ ]:
function delaunay_triangulation(points)::Triangulation
    points_map = collect(1:size(points, 1))
    triangle_array = Triangle.basic_triangulation(points, points_map)
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(points, edges, triangles)
end

In [ ]:
function draw_edges!(triangulation; color=:gray)
    @unpack points, edges = triangulation
    linesegments!(points[edges'[:], :], color=color)
end

In [ ]:
function draw_triangulation(triangulation)
    make_scene()
    draw_edges!(triangulation)
    draw_points!(triangulation.points)   
end

In [ ]:
del = delaunay_triangulation(points)
draw_triangulation(del)

In [ ]:
pointset_mean(array) = dropdims(mean(array, dims=2), dims=2)

In [ ]:
abstract type TriangleCenter end

struct TriangleCentroid <: TriangleCenter end
struct TriangleIncenter <: TriangleCenter end
struct TriangleCircumcenter <: TriangleCenter end # yield Voronoi points!

In [ ]:
function triangle_centers(triangulation, ::TriangleCentroid)
    @unpack points, triangles = triangulation
    return pointset_mean(points[triangles, :])
end

triangle_centers(t) = triangle_centers(t, TriangleCentroid())

In [ ]:
function triangle_centers(triangulation, ::TriangleIncenter)
    @unpack points, triangles = triangulation
    centers = []
    for t in eachrow(triangles)
        corners = points[t, :]

        a = norm(corners[2, :] - corners[3, :])
        b = norm(corners[1, :] - corners[3, :])
        c = norm(corners[1, :] - corners[2, :])
        # based on barycentric coordinates a:b:c        
        incenter = [a b c] * corners ./ (a + b + c)
        push!(centers, incenter)
    end
    
    return vcat(centers...)
end

In [ ]:
function triangle_centers(triangulation, ::TriangleCircumcenter)
    @unpack points, triangles = triangulation
    centers = []
    for t in eachrow(triangles)
        corners = points[t, :]
        Ax, Ay = corners[1, :]
        Bx, By = corners[2, :]
        Cx, Cy = corners[3, :]
        D = 2 * ( Ax * (By - Cy) + Bx * (Cy - Ay) + Cx * (Ay - By) )
        Ux = ((Ax^2 + Ay^2)*(By - Cy)) + ((Bx^2 + By^2)*(Cy - Ay)) + ((Cx^2 + Cy^2)*(Ay - By))
        Uy = ((Ax^2 + Ay^2)*(Cx - Bx)) + ((Bx^2 + By^2)*(Ax - Cx)) + ((Cx^2 + Cy^2)*(Bx - Ax))
        circumcenter = [Ux Uy] ./ D
        push!(centers, circumcenter)
    end
    
    return vcat(centers...)
end

In [ ]:
draw_triangulation(del)
draw_points!(triangle_centers(del, TriangleCentroid()), color=:limegreen)
draw_points!(triangle_centers(del, TriangleIncenter()), color=:magenta)
draw_points!(triangle_centers(del, TriangleCircumcenter()), color=:lightblue)

In [ ]:
function delaunay_with_centers(triangulation, method=TriangleCentroid())
    centers = triangle_centers(triangulation, method)
    all_points = vcat(triangulation.points, centers)
    return delaunay_triangulation(all_points)
end

In [ ]:
draw_triangulation(delaunay_with_centers(del))

In [ ]:
draw_triangulation(delaunay_with_centers(del, TriangleIncenter()))

In [ ]:
draw_triangulation(delaunay_with_centers(del, TriangleCircumcenter()))

In [ ]:
draw_triangulation(delaunay_with_centers(delaunay_with_centers(del)))

In [ ]:
function constrained_with_centers(triangulation, method=TriangleCentroid())
    @unpack points, edges = triangulation
    centers = triangle_centers(triangulation, method)
    all_points = vcat(points, centers)
    point_map = collect(1:size(all_points, 1))
    
    triangle_array = Triangle.constrained_triangulation(all_points, point_map, edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(all_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_with_centers(del))

In [ ]:
draw_triangulation(constrained_with_centers(constrained_with_centers(del)))

In [ ]:
function edge_midpoints(triangulation)
    @unpack points, edges = triangulation
    return pointset_mean(points[edges, :])
end

In [ ]:
draw_triangulation(del)
draw_points!(edge_midpoints(del), color=:red)

In [ ]:
# triangulate with edge subdivision, again
del_ = delaunay_triangulation(vcat(del.points, edge_midpoints(del)))
draw_triangulation(delaunay_triangulation(vcat(del_.points, edge_midpoints(del_))))

In [ ]:
function subdivided_edges(edges, offset)
    set = Vector()
    for e in 1:size(edges, 1)
        edge = edges[e, :]
        push!(set, [edge[1] e + offset])
        push!(set, [e + offset edge[2]])
    end
    return vcat(set...)
end

In [ ]:
function constrained_subdivision(triangulation)
    @unpack points, edges = triangulation
    new_points = vcat(points, edge_midpoints(triangulation))
    point_map = collect(1:size(new_points, 1))
    keep_edges = subdivided_edges(edges, size(points, 1))

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, keep_edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(new_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_subdivision(del))

In [ ]:
draw_triangulation(constrained_subdivision(constrained_subdivision(del)))

In [ ]:
# alternate: first add triangle centers, then subdivide edges
draw_triangulation(constrained_subdivision(constrained_with_centers(del)))

In [ ]:
# alternate 2: first subdivide edges, the add triangle centers
draw_triangulation(constrained_with_centers(constrained_subdivision(del)))

In [ ]:
# alternate 2.5: first subdivide edges, the add triangle centers (unconstrained)
draw_triangulation(delaunay_with_centers(constrained_subdivision(del)))

In [ ]:
function constrained_combined_refinement(triangulation, method=TriangleCentroid())
    @unpack points, edges = triangulation
    centers = triangle_centers(triangulation, method)
    new_points = vcat(points, edge_midpoints(triangulation), centers)
    point_map = collect(1:size(new_points, 1))
    keep_edges = subdivided_edges(edges, size(points, 1))

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, keep_edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(new_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_combined_refinement(del))

In [ ]:
draw_triangulation(constrained_combined_refinement(constrained_combined_refinement(del)))

In [ ]:
function combined_refinement(triangulation, method=TriangleCentroid())
    @unpack points, edges = triangulation
    centers = triangle_centers(triangulation, method)
    new_points = vcat(points, edge_midpoints(triangulation), centers)

    return delaunay_triangulation(new_points)
end

In [ ]:
draw_triangulation(combined_refinement(del))

In [ ]:
draw_triangulation(combined_refinement(combined_refinement(del)))

# Steiner Tree Model

In [ ]:
function antiparallel_digraph(triangulation)
    @unpack points, edges = triangulation
    graph = SimpleDiGraph(size(points, 1))
    for e in 1:size(edges, 1)
        s, t = edges[e, :]
        add_edge!(graph, s, t)
        add_edge!(graph, t, s)
    end
    return graph
end

In [ ]:
function edge_lengths(points, edges)
    diff = points[edges[:, 1], :] - points[edges[:, 2], :]
    return dropdims(mapslices(norm, diff, dims=2), dims=2)
end

edge_lengths(triangulation) = edge_lengths(triangulation.points, triangulation.edges)

In [ ]:
function edge_length_map(triangulation)
    @unpack edges = triangulation
    lengths = edge_lengths(triangulation)
    
    length_map = Dict{Tuple{Int64, Int64}, Float64}()
    for e = 1:size(edges, 1)
        s, t = edges[e, :]
        length_map[s, t] = lengths[e]
        length_map[t, s] = lengths[e]
    end
    return length_map
end

In [ ]:
function steiner_tree(triangulation, terminals)
    # Compute length by Euclidean distance of nodes.
    lengths = edge_length_map(triangulation)
    
    # Build digraph with all antiparallel arcs, for nonnegative flow.
    graph = antiparallel_digraph(triangulation)
    nodes = collect(1:nv(graph))
    arcs = collect(keys(lengths))
    
    length(terminals) >= 2 || error("Need at least 2 terminals.")
    all(terminals .<= nv(graph)) || error("Terminals out of range.")
    root = terminals[1]
    sinks = terminals[2:end]
   
    demand(v, s) = 1.0*(v == s) - 1.0*(v == root)
    
    # Using arc length for fixed capacity cost and multi-commodity flow.
    model = JuMP.direct_model(SCIP.Optimizer(display_verblevel=0))
    @variable(model, select[a in arcs], Bin, container=SparseAxisArray)
    @variable(model, flow[a in arcs,s in sinks] ≥ 0, container=SparseAxisArray)
    @constraint(model, balance[v in nodes, s in sinks],
        sum(flow[(n, v), s] - flow[(v, n), s] for n in neighbors(graph, v))
        == demand(v, s))
    @constraint(model, capacity[a in arcs, s in sinks], flow[a, s] <= select[a])
    @objective(model, Min, sum(lengths[a] * select[a] for a in arcs))
    
    optimize!(model)
    
    return objective_value(model), value.(select)

end

In [ ]:
function draw_tree(triangulation, terminals)
    @unpack points, edges = triangulation

    obj, select = steiner_tree(triangulation, terminals)
    @show obj
    selected = [select[(s,t)] + select[(t,s)] for (s,t) in eachrow(edges)]
    active_edges = edges[selected .> 0.0, :]
    
    draw_triangulation(triangulation)
    linesegments!(points[active_edges'[:], :], color=:plum, linewidth=3)
    draw_points!(points[terminals, :], markersize=5, color=:teal)
end

In [ ]:
terminals = collect(1:7);

In [ ]:
draw_tree(del, terminals)

In [ ]:
draw_tree(constrained_subdivision(del), terminals)

In [ ]:
draw_tree(delaunay_with_centers(del), terminals)

In [ ]:
draw_tree(constrained_with_centers(del, TriangleIncenter()), terminals)

In [ ]:
draw_tree(constrained_combined_refinement(del, TriangleIncenter()), terminals)

In [ ]:
draw_tree(constrained_combined_refinement(del, TriangleCircumcenter()), terminals)

In [ ]:
draw_tree(constrained_with_centers(del, TriangleCircumcenter()), terminals)

In [ ]:
draw_tree(combined_refinement(del, TriangleIncenter()), terminals)

In [ ]:
draw_tree(combined_refinement(del, TriangleCircumcenter()), terminals)

In [ ]:
draw_tree(
    combined_refinement(
        combined_refinement(del, TriangleIncenter()),
        TriangleIncenter()),
    terminals)

# Edge Triangle Incidence

We want to store the edge-triangle incidence explicitly, so that we can analyze neighboring triangles easily.

In [ ]:
function edge_triangle_incidence(triangulation::Triangulation)
    @unpack edges, triangles = triangulation
    incidence = spzeros(Bool, size(edges, 1), size(triangles, 1))
    edges_nested = collect(eachrow(edges))
    for (ti, t) in enumerate(eachrow(triangles))
        for edge in (min(t[[1, 2]], t[[2, 1]]),
                     min(t[[1, 3]], t[[3, 1]]),
                     min(t[[2, 3]], t[[3, 2]]))
            ei = findfirst(isequal(edge), edges_nested)
            incidence[ei, ti] = true
        end
    end
    return incidence
end

In [ ]:
# compute for delaunay triangulation
del_e2t = edge_triangle_incidence(del)
Array(del_e2t)

In [ ]:
# find edges that are incident to two triangles (not on boundary)
function inner_edges(incidence)
    range = 1:size(incidence, 1)
    cond = sum(incidence, dims=2) .== 2
    return range[cond[:]]
end
inner_edges(del_e2t)

In [ ]:
# find "opposing" points for each inner edge
function opposing_points(triangulation, incidence)
    @unpack edges, triangles = triangulation
    find_other_point(e, tri) = filter(p -> !(p in edges[e, :]), triangles[tri, :])[1]
    list = NTuple{3, Int}[]
    for e in inner_edges(incidence)
        (tri1, tri2), _ = findnz(incidence[e, :])
        point1 = find_other_point(e, tri1)
        point2 = find_other_point(e, tri2)
        push!(list, (e, point1, point2))
    end
    return list
end

In [ ]:
opposing_points(del, del_e2t)

In [ ]:
function graph(triangulation)
    @unpack points, edges = triangulation
    graph = SimpleGraph(size(points, 1))
    for e in 1:size(edges, 1)
        s, t = edges[e, :]
        add_edge!(graph, s, t)
    end
    return graph
end

In [ ]:
delg = graph(del)

In [ ]:
function distance_matrix(triangulation)
    @unpack points = triangulation
    return pairwise(Euclidean(), points, points, dims=1)
end

In [ ]:
deld = distance_matrix(del)

In [ ]:
"Shortest path distance between two opposing vertices of adjacent triangles."
function shortest_path(graph, s, t, distances)
    heur(n) = distances[n, t]
    path = a_star(graph, s, t, distances, heur)
    return sum(distances[src(edge), dst(edge)] for edge in path)
end

In [ ]:
shortest_path(delg, 1, 3, deld)

In [ ]:
for t in opposing_points(del, del_e2t)
    los = deld[t[2], t[3]]
    spd = shortest_path(delg, t[2], t[3], deld)
    @show t los spd spd-los
end

In [ ]:
draw_triangulation(del)
draw_points!(del.points[[2, 6], :], markersize=6, color=:orange)

In [ ]:
# fresh Delaunay with single edge midpoint added
draw_tree(delaunay_triangulation(vcat(del.points, edge_midpoints(del)[2:2, :])), 1:7)

In [ ]:
# better use constrained Delaunay with new point, or else we will lose good Delaunay edge, as above!

In [ ]:
"Constrained Delaunay Triangulation, keeping all edges, with one edge midpoint added."
function constrained_subdivision(triangulation, edge)
    @unpack points, edges = triangulation
    new_points = vcat(points, edge_midpoints(triangulation)[edge:edge, :])
    n = size(new_points, 1)
    point_map = collect(1:n)
    keep_edges = vcat(edges[1:edge-1, :], edges[edge+1:end, :],
                      [edges[edge, 1]  n], [edges[edge, 2] n])

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, keep_edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(new_points, edges, triangles)
end

In [ ]:
"CDT, keeping all edges, finding the best inner edge to subdivide."
function constrained_longest_subdivision(triangulation)
    incidence = edge_triangle_incidence(triangulation)
    g = graph(triangulation)
    dist = distance_matrix(triangulation)
    
    longest_edge, longest_shortcut = -1, -1.0
    for (e, p1, p2) in opposing_points(triangulation, incidence)
        shortcut = shortest_path(g, p1, p2, dist) - dist[p1, p2]
        if shortcut > longest_shortcut
            longest_edge = e
            longest_shortcut = shortcut
        end
    end
    
    return constrained_subdivision(triangulation, longest_edge)
end

In [ ]:
draw_tree(constrained_longest_subdivision(del), 1:7)

In [ ]:
# repeat!
draw_tree(constrained_longest_subdivision(
            constrained_longest_subdivision(
                del)), 1:7)

In [ ]:
# not bad, but midpoint subdivision:
# - does not improve shortest-path distance optimally?
# - looks too regular?
#
# other options:
# - intersect the subdivided edge with line-of-sight
# - some convex combination of intersection with midpoint!

In [ ]:
# iterations
draw_tree(constrained_longest_subdivision(
          constrained_longest_subdivision(
          constrained_longest_subdivision(
          constrained_longest_subdivision(
          constrained_longest_subdivision(
          constrained_longest_subdivision(
                del)))))), 1:7)

In [ ]:
# works OK, but keeping all edges is too strict:
# should only keep original (Delaunay) edges and their subdivision,
# not any new edges

In [ ]:
# for reference: original Delaunay Triangulation:
draw_tree(del, 1:7)

# Store edges to keep explicitly.

In [ ]:
function initial_delaunay(points)
    @unpack points, edges, triangles = delaunay_triangulation(points) 
    keep_all = collect(1:size(edges, 1))
    return Triangulation(points, edges, triangles, keep_all)
end

In [ ]:
"Constrained Delaunay Triangulation, keeping some edges, with one edge midpoint added."
function partially_constrained_subdivision(triangulation, edge, keep_new_edge=false)
    @unpack points, edges, keep_edges = triangulation
    new_points = vcat(points, edge_midpoints(triangulation)[edge:edge, :])
    n = size(new_points, 1)
    point_map = collect(1:n)
    
    # are we subdividing a protected edge?
    protected = if (edge in keep_edges) || keep_new_edge
        keep_edges2 = filter(!isequal(edge), keep_edges)
        vcat(edges[keep_edges2, :], [edges[edge, 1]  n], [edges[edge, 2] n])
    else
        edges[keep_edges, :]
    end

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, protected)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    
    # HACK: should store differently
    rows = collect(eachrow(protected))
    keep_edges2 = [e for e in 1:size(edges, 1) if edges[e, :] in rows]
    
    return Triangulation(new_points, edges, triangles, keep_edges2)
end

In [ ]:
"CDT, keeping all edges, finding the best inner edge to subdivide."
function constrained_longest_subdivision2(triangulation, keep_new_edge=false)
    incidence = edge_triangle_incidence(triangulation)
    g = graph(triangulation)
    dist = distance_matrix(triangulation)
    
    longest_edge, longest_shortcut = -1, -1.0
    for (e, p1, p2) in opposing_points(triangulation, incidence)
        shortcut = shortest_path(g, p1, p2, dist) - dist[p1, p2]
        if shortcut > longest_shortcut
            longest_edge = e
            longest_shortcut = shortcut
        end
    end
    
    return partially_constrained_subdivision(triangulation, longest_edge, keep_new_edge)
end

In [ ]:
del2 = initial_delaunay(points)

In [ ]:
draw_tree(constrained_longest_subdivision2(del2), 1:7)

In [ ]:
function repeatedly(refinement, times, input)
    for i in 1:times
        input = refinement(input)
    end
    return input
end

In [ ]:
draw_tree(repeatedly(constrained_longest_subdivision2, 2, del2), 1:7)

In [ ]:
draw_tree(repeatedly(constrained_longest_subdivision2, 4, del2), 1:7)

In [ ]:
draw_tree(repeatedly(constrained_longest_subdivision2, 8, del2), 1:7)

In [ ]:
draw_tree(repeatedly(constrained_longest_subdivision2, 16, del2), 1:7)

In [ ]:
draw_tree(repeatedly(constrained_longest_subdivision2, 32, del2), 1:7)

In [ ]:
"CDT, keeping all edges, finding the best inner edge to subdivide."
function constrained_longest_edge_subdivision(triangulation)
    longest_edge = argmax(edge_lengths(triangulation))
    return partially_constrained_subdivision(triangulation, longest_edge)
end

In [ ]:
draw_tree(repeatedly(constrained_longest_edge_subdivision, 1, del2), 1:7)

In [ ]:
draw_tree(repeatedly(constrained_longest_edge_subdivision, 16, del2), 1:7)

In [ ]:
draw_tree(del2, 1:7)

In [ ]:
# combine the two methods (shortcut, longest edge) in alternating manner
alternate(t) = constrained_longest_edge_subdivision(constrained_longest_subdivision2(t))
alternate2(t) = constrained_longest_edge_subdivision(constrained_longest_subdivision2(t, true))

In [ ]:
draw_tree(repeatedly(alternate, 1, del2), 1:7)

In [ ]:
draw_tree(repeatedly(alternate, 5, del2), 1:7)

In [ ]:
draw_tree(repeatedly(alternate2, 2, del2), 1:7)

In [ ]:
draw_tree(repeatedly(alternate2, 5, del2), 1:7)